In [23]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from helper import clean_column_names

In [24]:
### clean data nola gov tbl ### 
def clean_gov_table(df):
    df = df.applymap(lambda x: x.lower().strip() if isinstance(x, str) else x)

    df.loc[:, "tracking_id"] = (df["complaint_tracking_number"].str
                                .lower()
                                .str.strip()
                                .str.replace(r"\s+", "", regex=True)
                                .str.replace(r"\'", "", regex=True)
    )


    return df 

def read_gov_tbl():
    df = pd.read_csv("data/input/data_nola_gov.csv")
    return df 

def extract_year(df):
    year = df.tracking_id.str.extract(r"^(\w{4})")

    df.loc[:, "year"] = year[0].astype(str)
    return df


def read_brady_tbl():
    df = pd.read_csv("../brady/data/output/dv_df_brady.csv")
    return df 

In [25]:
dv_df_brady = read_brady_tbl()


dv_df_brady.allegation.value_counts(ascending=False)

allegation
rule 4: perf of duty paragraph 04 - neglect of duty                           193
rule 4: perf of duty paragraph 02 - instructions from authoritative source     23
rule 4: perf of duty paragraph 04 neglect of duty                               1
Name: count, dtype: int64

In [26]:

dv_df_brady.loc[:, "allegation_desc"] = dv_df_brady.allegation_desc.str.replace(r"sustained ", "", regex=True)
dv_df_brady.allegation_desc.value_counts(ascending=False)

allegation_desc
nopd policy: chapter 42.4 - domestic violence                                                                                                                 34
nopd policy: chapter 42.4 - domestic violence; paragraph 82: reports                                                                                          24
nopd policy: chapter 42.4 - domestic violence; paragraph 91                                                                                                   12
nopd policy: chapter 42.4 - domestic violence; paragraph 16                                                                                                   10
nopd policy: chapter 42.4 - domestic violence; paragraph 18                                                                                                    7
                                                                                                                                                              ..
nopd policy: chapt

In [27]:
dv_ids = [n for n in dv_df_brady["tracking_id"]]

gov_tbl = read_gov_tbl()
gov_tbl = gov_tbl.pipe(clean_column_names).pipe(clean_gov_table).pipe(extract_year)



/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_17005/3086914024.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [28]:


gov_tbl = gov_tbl[~((gov_tbl.incident_type.fillna("") == ""))]
gov_tbl = gov_tbl[~((gov_tbl.disposition.fillna("") == ""))]

gov_tbl = gov_tbl[gov_tbl.year.str.contains("2016|2017|2018|2019|2020|2021|2022|2023")]

gov_tbl.shape

# gov_tbl.to_csv("data/output/gov_tbl_complete.csv", index=False)

(7682, 26)

In [29]:
gov_tbl_domestic_violence = gov_tbl[gov_tbl.tracking_id.isin(dv_ids)]

# gov_tbl_domestic_violence.to_csv("data/output/gov_tbl_w_brady.csv")

gov_tbl_domestic_violence.tracking_id.nunique()



105

In [30]:
g
print(f"Before: {gov_tbl_domestic_violence.shape}")
gov_tbl_domestic_violence = gov_tbl_domestic_violence.drop_duplicates(subset=["tracking_id", "rule_violation", "paragraph_violation", "disposition", "unique_officer_allegation_id"])
print(f"After: {gov_tbl_domestic_violence.shape}")


NameError: name 'g' is not defined

In [ ]:

gov_tbl_domestic_violence.loc[:, "allegation"] = gov_tbl_domestic_violence.rule_violation.str.cat(gov_tbl_domestic_violence.paragraph_violation, sep=" ")


gov_tbl_domestic_violence.allegation.value_counts(ascending=False)

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_17005/1489970448.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



allegation
rule 4: perf of duty paragraph 04 - neglect of duty                           138
rule 4: perf of duty paragraph 02 - instructions from authoritative source      8
rule 3: prof conduct paragraph 01 - professionalism                             6
rule 6: official info paragraph 02 - false or inaccurate reports                4
rule 4: perf of duty paragraph 01 - reporting for duty                          2
rule 2: moral conduct paragraph 03 - honesty and truthfulness                   1
rule 5: rest activities paragraph 07 - acting impartially                       1
rule 2: moral conduct paragraph 02 - courtesy                                   1
Name: count, dtype: int64

In [ ]:

# Calculate the percentage of "sustained" dispositions for each year and incident type
public_initiated_pct = (gov_tbl[(gov_tbl.incident_type == "public initiated") & (gov_tbl.disposition == "sustained")].groupby("year").size() / 
                        gov_tbl[gov_tbl.incident_type == "public initiated"].groupby("year").size()) * 100

rank_initiated_pct = (gov_tbl[(gov_tbl.incident_type == "rank initiated") & (gov_tbl.disposition == "sustained")].groupby("year").size() / 
                      gov_tbl[gov_tbl.incident_type == "rank initiated"].groupby("year").size()) * 100

# Create the traces for the bar chart
trace1 = go.Bar(
    x=public_initiated_pct.index,
    y=public_initiated_pct,
    name='Public Initiated',
    marker_color='blue',
    opacity=0.8,
    text=[f"{pct:.2f}%" for pct in public_initiated_pct],
    textposition='auto',
    textangle=0
)

trace2 = go.Bar(
    x=rank_initiated_pct.index,
    y=rank_initiated_pct,
    name='Rank Initiated',
    marker_color='green',
    opacity=0.8,
    text=[f"{pct:.2f}%" for pct in rank_initiated_pct],
    textposition='auto',
    textangle=0
)

# Create the layout for the chart
layout = go.Layout(
    title='Percentage of Sustained Dispositions by Incident Type and Year',
    xaxis=dict(title='Year', tickangle=0),
    yaxis=dict(title='Percentage of Sustained Dispositions'),
    barmode='group'
)

# Create the figure and display the chart
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

In [ ]:
# Calculate the percentage of "sustained" dispositions for each year and incident type
public_initiated_pct_dv = (gov_tbl_domestic_violence[(gov_tbl_domestic_violence.incident_type == "public initiated") & (gov_tbl.disposition == "sustained")].groupby("year").size() / 
                        gov_tbl_domestic_violence[gov_tbl_domestic_violence.incident_type == "public initiated"].groupby("year").size()) * 100

rank_initiated_pct_dv = (gov_tbl_domestic_violence[(gov_tbl_domestic_violence.incident_type == "rank initiated") & (gov_tbl.disposition == "sustained")].groupby("year").size() / 
                      gov_tbl_domestic_violence[gov_tbl_domestic_violence.incident_type == "rank initiated"].groupby("year").size()) * 100

# Create the traces for the bar chart
trace1 = go.Bar(
    x=public_initiated_pct_dv.index,
    y=public_initiated_pct_dv,
    name='Public Initiated',
    marker_color='blue',
    opacity=0.8,
    text=[f"{pct:.2f}%" for pct in public_initiated_pct_dv],
    textposition='auto',
    textangle=0
)

trace2 = go.Bar(
    x=rank_initiated_pct_dv.index,
    y=rank_initiated_pct_dv,
    name='Rank Initiated',
    marker_color='green',
    opacity=0.8,
    text=[f"{pct:.2f}%" for pct in rank_initiated_pct_dv],
    textposition='auto',
    textangle=0
)

# Create the layout for the chart
layout = go.Layout(
    title='Percentage of Sustained Dispositions by Incident Type and Year for Domestic Violence Subset',
    xaxis=dict(title='Year', tickangle=0),
    yaxis=dict(title='Percentage of Sustained Dispositions'),
    barmode='group'
)

# Create the figure and display the chart
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_17005/1739113930.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_17005/1739113930.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [ ]:
def read_calls_for_service_tbl():
    df = pd.read_csv("../gov-cfs/data/output/calls_for_service_2016_2023.csv")
    return df 

calls_for_service_tbl = read_calls_for_service_tbl()

# Calculate the counts for filtered data
public_initiated_counts = gov_tbl_domestic_violence[gov_tbl_domestic_violence.incident_type == "public initiated"].groupby("year").size()
rank_initiated_counts = gov_tbl_domestic_violence[gov_tbl_domestic_violence.incident_type == "rank initiated"].groupby("year").size()

# Calculate percent changes for filtered data
public_initiated_pct_change = public_initiated_counts.pct_change() * 100
rank_initiated_pct_change = rank_initiated_counts.pct_change() * 100

# Calculate the counts for all incidents
all_incident_counts = gov_tbl_domestic_violence.groupby("year").size()

# Calculate percent changes for all incidents
all_incident_pct_change = all_incident_counts.pct_change() * 100

def format_labels(count, pct):
    if not pd.isna(pct):
        return f'{count}<br>({pct:.1f}%)'
    else:
        return f'{count}'
    
    
# Calculate the counts for calls for service data
calls_for_service_counts = calls_for_service_tbl.groupby("year").size()

# Calculate percent changes for calls for service data
calls_for_service_pct_change = calls_for_service_counts.pct_change() * 100

# Create subplots for each incident chart
fig1 = make_subplots(rows=2, cols=1, subplot_titles=("Public Initiated Incidents", "Calls for Service"), vertical_spacing=0.15)
fig2 = make_subplots(rows=2, cols=1, subplot_titles=("Rank Initiated Incidents", "Calls for Service"), vertical_spacing=0.15)
fig3 = make_subplots(rows=2, cols=1, subplot_titles=("Domestic Allegations", "Calls for Service"), vertical_spacing=0.15)

# Add traces for incident data
fig1.add_trace(go.Scatter(
    x=public_initiated_counts.index,
    y=public_initiated_counts,
    mode='lines+markers+text',
    name='Public Initiated',
    marker=dict(color='yellow'),
    text=[format_labels(count, pct) for count, pct in zip(public_initiated_counts, public_initiated_pct_change)],
    textposition='top center',
    textfont=dict(size=10),
    texttemplate='%{text}',  # Add this line to display the formatted text
    cliponaxis=False  # Add this line to prevent labels from being clipped
), row=1, col=1)

fig2.add_trace(go.Scatter(
    x=rank_initiated_counts.index,
    y=rank_initiated_counts,
    mode='lines+markers+text',
    name='Rank Initiated',
    marker=dict(color='green'),
    text=[format_labels(count, pct) for count, pct in zip(rank_initiated_counts, rank_initiated_pct_change)],
    textposition='top center',
    textfont=dict(size=10),
    texttemplate='%{text}',  # Add this line to display the formatted text
    cliponaxis=False  # Add this line to prevent labels from being clipped
), row=1, col=1)

fig3.add_trace(go.Scatter(
    x=all_incident_counts.index,
    y=all_incident_counts,
    mode='lines+markers+text',
    name='All Incidents',
    marker=dict(color='red'),
    text=[format_labels(count, pct) for count, pct in zip(all_incident_counts, all_incident_pct_change)],
    textposition='top center',
    textfont=dict(size=10),
    texttemplate='%{text}',  # Add this line to display the formatted text
    cliponaxis=False  # Add this line to prevent labels from being clipped
), row=1, col=1)

# Add traces for calls for service data
fig1.add_trace(go.Scatter(
    x=calls_for_service_counts.index,
    y=calls_for_service_counts,
    mode='lines+markers+text',
    name='Domestic Calls for Service',
    marker=dict(color='blue'),
    text=[format_labels(count, pct) for count, pct in zip(calls_for_service_counts, calls_for_service_pct_change)],
    textposition='top center',
    textfont=dict(size=10),
    texttemplate='%{text}',  # Add this line to display the formatted text
    cliponaxis=False  # Add this line to prevent labels from being clipped
), row=2, col=1)

fig2.add_trace(go.Scatter(
    x=calls_for_service_counts.index,
    y=calls_for_service_counts,
    mode='lines+markers+text',
    name='Domestic Calls for Service',
    marker=dict(color='blue'),
    text=[format_labels(count, pct) for count, pct in zip(calls_for_service_counts, calls_for_service_pct_change)],
    textposition='top center',
    textfont=dict(size=10),
    texttemplate='%{text}',  # Add this line to display the formatted text
    cliponaxis=False  # Add this line to prevent labels from being clipped
), row=2, col=1)

fig3.add_trace(go.Scatter(
    x=calls_for_service_counts.index,
    y=calls_for_service_counts,
    mode='lines+markers+text',
    name='Domestic Calls for Service',
    marker=dict(color='blue'),
    text=[format_labels(count, pct) for count, pct in zip(calls_for_service_counts, calls_for_service_pct_change)],
    textposition='top center',
    textfont=dict(size=10),
    texttemplate='%{text}',  # Add this line to display the formatted text
    cliponaxis=False  # Add this line to prevent labels from being clipped
), row=2, col=1)

# Update layout for each subplot
fig1.update_layout(
    title='Total Counts and Percent Change by Year for Public Initiated Domestic Allegations',
    xaxis=dict(title='Year', tickangle=0),
    yaxis=dict(title='Count'),
    xaxis2=dict(title='Year', tickangle=0),
    yaxis2=dict(title='Calls for Service Count'),
    height=800,  # Increase the height of the plot
    width=1000   # Increase the width of the plot
)

fig2.update_layout(
    title='Total Counts and Percent Change by Year for Rank Initiated Domestic Allegations',
    xaxis=dict(title='Year', tickangle=0),
    yaxis=dict(title='Count'),
    xaxis2=dict(title='Year', tickangle=0),
    yaxis2=dict(title='Calls for Service Count'),
    height=800,  # Increase the height of the plot
    width=1000   # Increase the width of the plot
)

fig3.update_layout(
    title='Total Counts and Percent Change by Year for Domestic Allegations',
    xaxis=dict(title='Year', tickangle=0),
    yaxis=dict(title='Count'),
    xaxis2=dict(title='Year', tickangle=0),
    yaxis2=dict(title='Calls for Service Count'),
    height=800,  # Increase the height of the plot
    width=1000   # Increase the width of the plot
)

# Show the charts
fig3.show()
fig1.show()
fig2.show()

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_17005/1322337499.py:2: DtypeWarning:

Columns (1,23) have mixed types. Specify dtype option on import or set low_memory=False.



hello

Count by officer accused vs something else like failing to investigate